In [2]:
from scipy.spatial.distance import cdist
import numpy as np

demo_data = np.array([[5, 4, 4, 3, 5], [3, 4, 2, 5, 3], [2, 1, 2, 2, 5], [2, 0, 3, 1, 3],])
misha = np.array([[0, 4, 4, 4, 5]])
cdist(demo_data, misha, metric='cosine')

array([[0.15342203],
       [0.12999789],
       [0.14560384],
       [0.24345225]])

7. Матричная факторизация и факторизационные машины. Практика

In [1]:
!pip install implicit 

   ---------------------------------------- 0.0/750.8 kB ? eta -:--:--
   --------------------------------------- 750.8/750.8 kB 10.4 MB/s eta 0:00:00


In [5]:
import numpy as np
import scipy
import pandas as pd

from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k
from sklearn.model_selection import train_test_split

In [3]:
ratings = pd.read_csv("data/ml-100k/u.data", sep="\t", header=None)
ratings.columns = ['user_id', 'item_id', 'rating', 'timestamp']
ratings.sort_values('timestamp', inplace=True)
ratings['score'] = (ratings['rating'] > 2).apply(int)

In [6]:
train, test = train_test_split(ratings, test_size=0.2, shuffle=False)

In [7]:
train_pivot = pd.pivot_table(
    train,
    index="user_id", 
    columns="item_id", 
    values="score"
)
test_pivot = pd.pivot_table(
    test,
    index="user_id", 
    columns="item_id", 
    values="score"
)

print(train_pivot.shape)
print(test_pivot.shape)
## (751, 1616)
## (301, 1448)

(751, 1616)
(301, 1448)


Теперь создадим сводную таблицу из таблицы rating, заполнив её ячейки нулями. Получим матрицу размером 943 x 1682. Для тех фильмов, которым пользователь выставил оценку значения, будут равны 0, для остальных — пропуску. 

In [8]:
shell = pd.pivot_table(
    ratings, 
    index="user_id", 
    columns="item_id", 
    values="score", 
    aggfunc=lambda x: 0
)
shell.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Примечание. Сложение DataFrame производится на пересечении соответствующих индексов строк и столбцов. Элементы, которые не попали в пересечение помечаются как пропуски. 

In [9]:
train_pivot = shell + train_pivot
test_pivot = shell + test_pivot

train_pivot = (train_pivot + 1).fillna(0)
test_pivot = (test_pivot + 1).fillna(0)
print(train_pivot.shape)
print(test_pivot.shape)
## (943, 1682)
## (943, 1682)

train_pivot.head()

(943, 1682)
(943, 1682)


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,2.0,2.0,2.0,0.0,2.0,2.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
train_pivot_sparse = scipy.sparse.csr_matrix(train_pivot.values)
test_pivot_sparse = scipy.sparse.csr_matrix(test_pivot.values)

In [11]:
model = AlternatingLeastSquares(factors=10, random_state=42)
model.fit(train_pivot_sparse)

c:\Projects\DS\lessons\.venv\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

Номера столбцов начинаются от 0, а вот идентификаторы фильмов в нашем примере выражаются числами и начинаются от 1. Поэтому необходимо сначала выделить все уникальные идентификаторы товаров в отдельный вектор, а уже затем обращаться к нему по полученным индексам.

In [12]:
unique_items = np.array(train_pivot.columns)
user_id = 14
recomendations_ids, scores = model.recommend(user_id, train_pivot_sparse[user_id])
recomendations = unique_items[recomendations_ids]
print('Recomendations ids: {}'.format(recomendations_ids))
print('Recomendations for user {}: {}'.format(user_id, recomendations))

## Recomendations ids: [293 116  99 275 244 287 283 150 125 596]
## Recomendations for user 14: [294 117 100 276 245 288 284 151 126 597]

Recomendations ids: [293 116 244 275 287  99 283 312 596 150]
Recomendations for user 14: [294 117 245 276 288 100 284 313 597 151]


In [16]:
movies = pd.read_csv('data/movies.zip')
for i in recomendations:
    print(movies[movies.movieId==i].title)

255    Perez Family, The (1995)
Name: title, dtype: object
102    Young Poisoner's Handbook, The (1995)
Name: title, dtype: object
Series([], Name: title, dtype: object)
238    Milk Money (1994)
Name: title, dtype: object
249    Natural Born Killers (1994)
Name: title, dtype: object
88    City Hall (1996)
Name: title, dtype: object
246    New York Cop (Nyû Yôku no koppu) (1993)
Name: title, dtype: object
272    Swan Princess, The (1994)
Name: title, dtype: object
514    Pretty Woman (1990)
Name: title, dtype: object
124    Rob Roy (1995)
Name: title, dtype: object


In [17]:
map_at10 = mean_average_precision_at_k(model, train_pivot_sparse, test_pivot_sparse, K=10)
print('Mean Average Precision at 10: {:.3f}'.format(map_at10))

## Mean Average Precision at 10: 0.087

  0%|          | 0/301 [00:00<?, ?it/s]

Mean Average Precision at 10: 0.084


In [18]:
!pip install lightfm

In [19]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

c:\Projects\DS\lessons\.venv\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [20]:
model = LightFM(no_components=10, loss='logistic', random_state=42)
model.fit(train_pivot_sparse, epochs=30)

In [ ]:
item_ids = np.arange(0, train_pivot_sparse.shape[1])
list_pred = model.predict(user_id, item_ids)
recomendations_ids = np.argsort(-list_pred)[:10]
recomendations = unique_items[recomendations_ids]
print('Recomendations for user {}: {}'.format(user_id, recomendations))

## Recomendations for user 14: [ 50 294 258 100 181 288 286   1 300 121]

In [21]:
map_at10 = precision_at_k(model, test_pivot_sparse, k=10).mean()
print('Mean Average Precision at 10: {:.2f}'.format(map_at10))
## Mean Average Precision at 10: 0.32

Mean Average Precision at 10: 0.32
